# Đồ án 2: Image Processing

## Thư viện và hình ảnh

### Import thư viện

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

### Đọc ảnh và chuyển về `np.ndarray`

In [ ]:
input_file = 'tiger.png'
image = Image.open(input_file)
image = np.array(image)

input_file_2 = 'tiger.png'
image_2 = Image.open(input_file_2)
image_2 = np.array(image_2)

## Thay đổi độ sáng

In [ ]:
def adjust_brightness(img, brightness):
    return np.uint8(np.clip(img + np.array([brightness], dtype=np.int16), 0, 255))

### Test trên ảnh

In [ ]:
# Brightness: -255 tới 255
brightness = -128
result = adjust_brightness(image, brightness)

In [ ]:
# Show ảnh trên notebook
plt.imshow(result)

In [ ]:
# Xuất ra file
output_file = input_file.split('.')[0] + '_' + str(brightness) + '_brightness' + '.png'
Image.fromarray(result).save(output_file)

## Thay đổi độ tương phản

In [ ]:
def adjust_contrast(img, contrast):
    contrast = np.clip(float(contrast), -255, 255)
    factor = (259 * (contrast + 255)) / (255 * (259 - contrast))
    return np.uint8(np.clip(factor * (img.astype(float) - 128) + 128, 0, 255))

### Test trên ảnh

In [ ]:
# Contrast: -255 tới 255
contrast = -128
result = adjust_contrast(image, contrast)

In [ ]:
# Show ảnh trên notebook
plt.imshow(result)

In [ ]:
# Xuất ra file
output_file = input_file.split('.')[0] + '_' + str(contrast) + '_contrast' + '.png'
Image.fromarray(result).save(output_file)

## Chuyển đổi ảnh RGB thành ảnh xám

In [ ]:
def to_grayscale(img, weight):
    return np.uint8(np.dot(img[..., :3], weight))

### Test trên ảnh

In [ ]:
# Weight: Dưới đây là công thức luma của các ảnh thuộc format CCIR 601
weight = [0.299 , 0.587, 0.114]
result = to_grayscale(image, weight)

In [ ]:
# Show ảnh trên notebook
    # cmap='gray': để show ảnh với 3 kênh màu khi ảnh xám chỉ có duy nhất 1 kênh
plt.imshow(result, cmap='gray')

In [ ]:
# Xuất ra file
output_file = input_file.split('.')[0] + '_grayscale' + '.png'
Image.fromarray(result).save(output_file, cmap='gray')

## Lật ảnh ngang - dọc

In [ ]:
def flip(img, direction):
    return np.flipud(img) if direction == 'vertical' else np.fliplr(img)

### Test trên ảnh

In [ ]:
# Direction: 'vertical' hoặc 'horizontal'
direction = 'vertical'
result = flip(image, direction)

In [ ]:
# Show ảnh trên notebook
plt.imshow(result)

In [ ]:
# Xuất ra file
output_file = input_file.split('.')[0] + '_flip_' + direction + '.png'
Image.fromarray(result).save(output_file)

## Chồng hai ảnh cùng kích thước

In [ ]:
def blend(img_1, img_2, alpha):
    return np.uint8(alpha * img_1.astype(float) + (1 - alpha) * img_2.astype(float))

### Test trên ảnh

In [ ]:
# Chuyển input thành ảnh xám, tuy nhiên vẫn có thể thực hiện được trên ảnh màu
gray_image_1 = to_grayscale(image, weight)
gray_image_2 = flip(to_grayscale(image_2, weight), 'horizontal')

In [ ]:
# Alpha: 0.0 tới 1.0
alpha = 0.5
result = blend(gray_image_1, gray_image_2, alpha)

In [ ]:
# Show ảnh trên notebook
plt.imshow(result, cmap='gray')

In [ ]:
# Xuất ra file
output_file = input_file.split('.')[0] + '_' + input_file_2.split('.')[0] + '_' + str(alpha) + '_blend' + '.png'
Image.fromarray(result).save(output_file, cmap='gray')

## Làm mờ ảnh

### Hàm Gaussian (còn gọi là Phân phối chuẩn trong thống kê)

In [ ]:
def Gaussian_func(x, sigma):
    return np.array(1 / (np.sqrt(2 * np.pi) * sigma) * (np.exp(-np.power(x / sigma, 2) / 2)))

### Tìm Gaussian filter kernel

In [ ]:
def calc_Gaussian_kernel(kernel_size, sigma):
    kernel_1d = np.linspace(-(kernel_size // 2), kernel_size // 2, num=kernel_size)
    kernel_1d = Gaussian_func(kernel_1d, sigma)
    kernel_2d = np.outer(kernel_1d.T, kernel_1d.T)
    kernel_2d *= 1.0 / np.sum(kernel_2d)
    return kernel_2d 

### Tích chập trên ma trận 2 chiều

In [ ]:
def convolve_layer(layer, kernel):
    view = kernel.shape + tuple(np.subtract(layer.shape, kernel.shape) + 1)
    submatrices = np.lib.stride_tricks.as_strided(layer, shape = view, strides = layer.strides * 2)
    return np.einsum('ij,ijkl->kl', kernel, submatrices)

### Tích chập trên ảnh màu gồm 3 kênh RGB

In [ ]:
def convolution(img, kernel):
    return np.dstack((convolve_layer(img[:,:,0], kernel), convolve_layer(img[:,:,1], kernel), convolve_layer(img[:,:,2], kernel)))

### Gaussian blur với kernel size tùy chọn

In [ ]:
def Gaussian_blur(img, kernel_size):
    kernel = calc_Gaussian_kernel(kernel_size, sigma=(kernel_size-1)/6)
    return np.uint8(convolution(img, kernel))

### Test trên ảnh

In [ ]:
# Kernel size: từ 1 đến dương vô cùng, phải là số lẻ
kernel_size = 15
result = Gaussian_blur(image, kernel_size)

In [ ]:
# Show ảnh trên notebook
plt.imshow(result)

In [ ]:
# Xuất ra file
output_file = input_file.split('.')[0] + '_' + str(kernel_size) + '_blur' + '.png'
Image.fromarray(result).save(output_file)